Import needed librarys

In [2]:
import os
current_directory = os.getcwd()
honey_page_url = "http://localhost:8000/honey_page.html"
import difflib
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

Configure the WebDriver

In [3]:
options = Options()
extension_path = os.path.join(current_directory,"extension","convert_URL_text_to_links")
#extension_path = os.path.join(current_directory,"extension","demo-extension")
options.add_argument(f"--load-extension={extension_path}")
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

Load and inject the mutation Observer into Selenium instance

In [4]:
with open("mutationObserver.js", "r") as file:
    mutation_observer_script = file.read()

driver.execute_cdp_cmd(
    "Page.addScriptToEvaluateOnNewDocument", {"source": mutation_observer_script}
)

{'identifier': '2'}

Load the honey page

In [5]:
driver.get(honey_page_url)
time.sleep(5)

Show changes introduced by Extension

In [6]:
mutation_logs = driver.execute_script("return window.mutationLogs;")
dom_content = driver.page_source
driver.quit()

for log in mutation_logs:
    print("Mutation type:", log["type"])
    print("Target element:", log["target"])
    if log["type"] == "childList":
        print("Added nodes:", log["addedNodes"])
        print("Removed nodes:", log["removedNodes"])
    elif log["type"] == "attributes":
        print("Changed attribute:", log["attributeName"])
    print("-" * 30)


Mutation type: childList
Target element: <p><a href="https://fakelogin.com" style="color: inherit !important;">https://fakelogin.com</a></p>
Added nodes: ['<a href="https://fakelogin.com" style="color: inherit !important;">https://fakelogin.com</a>']
Removed nodes: [None]
------------------------------


Save baseline DOM and final DOM to a file

In [7]:
dom_dump_path = os.path.join(current_directory,"DOM_DUMPS")
os.makedirs(dom_dump_path, exist_ok=True)

# get Baseline DOM
base_options = Options()
base_options.add_argument("--headless")
base_driver = webdriver.Chrome(options=base_options)
base_driver.get(honey_page_url)
baseline_DOM = base_driver.page_source
base_driver.quit()

with open(os.path.join(dom_dump_path,"baseline.html"), "w", encoding="utf-8") as file:
    file.write(baseline_DOM)

with open(os.path.join(dom_dump_path,"final.html"), "w", encoding="utf-8") as file:
    file.write(dom_content)


Compare baseline DOM with final DOM

In [8]:
diff = difflib.ndiff(baseline_DOM, dom_content)
addedLines = ''.join(x[2:] for x in diff if x.startswith('+ '))
print(addedLines)

<a href="" style="color: inherit !important;">https:fakelogin.com</a></p>
    </
